In [1]:
# Okay, let's preprocess all data

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf

2025-07-28 10:59:35.565284: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-28 10:59:35.600735: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-28 10:59:35.600762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-28 10:59:35.601590: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-28 10:59:35.607248: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# What all do we have? We have the geenrated features, and the densities defined for all the test blocks.

In [4]:
# Do the data prep for one block, and define a function for the rest

In [5]:
# densities are here
block_0103_density_file = np.load(os.path.join("data/densities_non_overlapping/", "all_densities_block_0103.npy"))

In [6]:
block_0103_density_file.shape

(910, 20, 1)

In [7]:
# Where are the feature files?

In [8]:
# for test time points
block_0103_feature_file_test_points = np.load(os.path.join("data/predicted_sequences_from_stage_1_non_overlapping/model_1/", "pred_values_blk_0103.npy"))

In [9]:
block_0103_feature_file_test_points.shape

(910, 7, 32)

In [10]:
# for train time points
block_0103_feature_file_train_points = np.load(os.path.join("data/train_features_non_overlapping/", "train_features_block_0103.npy"))

In [11]:
block_0103_feature_file_train_points.shape

(910, 13, 32)

In [12]:
# Stack the featrue files together?

In [13]:
block_0103_features = np.hstack((block_0103_feature_file_train_points, block_0103_feature_file_test_points))

In [14]:
block_0103_features.shape

(910, 20, 32)

In [15]:
# alternative way
block_0103_features_alt = np.concatenate((block_0103_feature_file_train_points, block_0103_feature_file_test_points), axis = 1)

In [16]:
block_0103_features_alt.shape

(910, 20, 32)

In [17]:
# sanity check
np.mean(block_0103_features == block_0103_features_alt)

1.0

In [18]:
# now stack this with the response?

In [19]:
block_0103_all_data = np.concatenate((block_0103_features, block_0103_density_file), axis = 2)

In [20]:
block_0103_all_data.shape

(910, 20, 33)

In [21]:
block_0103_all_data_alt = np.dstack((block_0103_features, block_0103_density_file))

In [22]:
block_0103_all_data_alt.shape

(910, 20, 33)

In [23]:
# sanity check
np.mean(block_0103_all_data_alt == block_0103_all_data)

1.0

In [24]:
# Okay, now need to create dfs, and store so these can be later used for the BLAR model

In [25]:
block_0103_all_data_df_100 = pd.DataFrame(block_0103_all_data[100,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density'])

In [26]:
%%time
counter = 0
for row in range(block_0103_all_data.shape[0]):
    # create the df
    current_df = pd.DataFrame(block_0103_all_data[row,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density'])
    # name the df
    df_name = 'block_0103_df_' + str(counter) + '.csv'
    # df path
    df_path = "data/BLAR_ready_dfs/model_1"
    # save the df
    current_df.to_csv(os.path.join(df_path, df_name), index = False)
    counter = counter + 1

CPU times: user 1.66 s, sys: 109 ms, total: 1.76 s
Wall time: 6.67 s


In [27]:
# block_0103_all_data_df_100

In [28]:
# Seems this is what we need, but do sanity checks

In [29]:
# Okay, how do we ensure we have done this correct

In [30]:
block_0103_features.shape

(910, 20, 32)

In [31]:
# sanity check for features
# train tps
print(np.mean(block_0103_features[:,:13,:] == block_0103_feature_file_train_points))
# test tps
print(np.mean(block_0103_features[:,13:,:] == block_0103_feature_file_test_points))

1.0
1.0


In [32]:
# Also check a few random dfs
block_0103_all_data_df_100 = pd.DataFrame(block_0103_all_data[100,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density']) # matches with the stored df
block_0103_all_data_df_500 = pd.DataFrame(block_0103_all_data[500,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density']) # matches with the stored df
block_0103_all_data_df_785 = pd.DataFrame(block_0103_all_data[785,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density']) # matches with the stored df
block_0103_all_data_df_91 = pd.DataFrame(block_0103_all_data[91,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density']) # matches with the stored df

In [33]:
# block_0103_all_data_df_91

In [34]:
# Okay, define a function to be used for the rest of the blocks

In [35]:
# What all do we need for the function?

# density_file_loc
# feature_file_loc_train
# feature_file_loc_test


In [36]:
def store_BLAR_ready_dfs(density_file_loc, feature_file_loc_train, feature_file_loc_test, block_name_for_df, save_df_path):
    # load density file
    density_array = np.load(density_file_loc)
    # load train features
    train_features = np.load(feature_file_loc_train)
    # load test featrues
    test_features = np.load(feature_file_loc_test)
    # stack features files
    stacked_features = np.hstack((train_features, test_features))
    # do sanity check
    stacked_features_alt = np.concatenate((train_features, test_features), axis = 1)
    print(np.mean(stacked_features == stacked_features_alt))
    # print shape
    print(stacked_features.shape)
    # stack features and density
    all_data_array = np.concatenate((stacked_features, density_array), axis = 2)
    # let's also do teh sanity checks
    all_data_alt = np.dstack((stacked_features, density_array))
    print(np.mean(all_data_array == all_data_alt))
    print(all_data_array.shape)
    # do the for loop and store the dfs

    catch_dfs = []
    counter = 0
    for row in range(all_data_array.shape[0]):
        # create the df
        current_df = pd.DataFrame(all_data_array[row,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density'])
        catch_dfs.append(current_df)
        # name the df
        df_name = block_name_for_df + str(counter) + '.csv'
        # save the df
        current_df.to_csv(os.path.join(save_df_path, df_name), index = False)
        counter = counter + 1

    return catch_dfs

In [37]:
# Try this for block 0103 - we need to create an additional folder for the block names

In [38]:
# argument values
dense_file_loc = "data/densities_non_overlapping/all_densities_block_0103.npy"
train_features_loc = "data/train_features_non_overlapping/train_features_block_0103.npy"
test_features_loc = "data/predicted_sequences_from_stage_1_non_overlapping/model_1/pred_values_blk_0103.npy"
blk_name = "block_0103_df_"
save_df_loc = "data/BLAR_ready_dfs/model_1/block_0103"

In [39]:
try_0103 = store_BLAR_ready_dfs(dense_file_loc, train_features_loc, test_features_loc, blk_name, save_df_loc)

1.0
(910, 20, 32)
1.0
(910, 20, 33)


In [40]:
# are the all the same?

# list files in "data/BLAR_ready_dfs/model_1" for block 0103

In [41]:
all_files_here = [file for file in os.listdir("data/BLAR_ready_dfs/model_1")]

In [42]:
len(all_files_here)

912

In [43]:
only_csv  = [file for file in os.listdir("data/BLAR_ready_dfs/model_1") if file.split(".")[-1] == 'csv']

In [44]:
# only_csv

In [45]:
len(only_csv)

910

In [46]:
# files inside the block_0103 folder
only_csv_block_0103 = [file for file in os.listdir(save_df_loc)]

In [47]:
len(only_csv_block_0103)

911

In [48]:
csv_files = ['block_0103_df_' + str(i) + '.csv' for i in range(910)]

In [49]:
# csv_files

In [50]:
%%time
catch_true = []
for file in csv_files:
    load_from_loc1 = pd.read_csv(os.path.join("data/BLAR_ready_dfs/model_1", file))
    load_from_loc2 = pd.read_csv(os.path.join(save_df_loc, file))
    is_true = load_from_loc1.equals(load_from_loc2)
    catch_true.append(is_true)

CPU times: user 1.82 s, sys: 32.2 ms, total: 1.86 s
Wall time: 1.88 s


In [51]:
np.mean(catch_true)

1.0

In [52]:
# Okay, should we delete the dfs outside of the blocked folders? - maybe let's leave them be for now

In [53]:
# Let's preprocess data for teh rest of the blocks